In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from shutil import copyfile

In [2]:
path = '../input/Plant_Village_Dataset/'

## NN

In [8]:
mean_sub = np.array([123.68, 116.779, 103.939], dtype=np.float32)
pre_processing = lambda x: (x - mean_sub)[:,:,:,::-1]

In [13]:
gen = ImageDataGenerator()

In [14]:
train_generator = gen.flow_from_directory(path+'train', target_size=(224,224), batch_size=64)
validation_generator = gen.flow_from_directory(path+'valid', target_size=(224,224), batch_size=64)

Found 1887 images belonging to 4 classes.
Found 830 images belonging to 4 classes.


In [15]:
training_samples = train_generator.n
validation_samples = validation_generator.n

training_samples, validation_samples

(1887, 830)

In [11]:
# model = Sequential([
#         BatchNormalization(axis=1, input_shape=(224,224,3)),
#         Conv2D(32, (3,3), activation='relu'),
#         Conv2D(32, (3,3), activation='relu'),
#         BatchNormalization(axis=1),
#         MaxPooling2D(),
        
#         Conv2D(64, (3,3), activation='relu'),
#         Conv2D(64, (3,3), activation='relu'),
#         BatchNormalization(axis=1),
#         MaxPooling2D(),
        
#         Flatten(),
        
#         Dense(256, activation='relu'),
#         BatchNormalization(),
#         Dropout(0.5),
#         Dense(4, activation='softmax')
#     ])


model = Sequential()

model.add(Lambda(pre_processing, input_shape=(224,224,3), 
                 output_shape=(224,224,3)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1000, activation='softmax'))

model.load_weights('vgg16_weights_tf_dim_ordering_tf_kernels.h5')

In [12]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_3 (Lambda)                (None, 224, 224, 3)   0           lambda_input_3[0][0]             
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 226, 226, 3)   0           lambda_3[0][0]                   
____________________________________________________________________________________________________
conv1_1 (Convolution2D)          (None, 224, 224, 64)  1792        zeropadding2d_2[0][0]            
____________________________________________________________________________________________________
zeropadding2d_3 (ZeroPadding2D)  (None, 226, 226, 64)  0           conv1_1[0][0]                    
___________________________________________________________________________________________

In [16]:
model.pop()

for l in model.layers:
    l.trainable = False

model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.fit_generator(train_generator, samples_per_epoch=training_samples, nb_epoch=5, 
                    validation_data=validation_generator, nb_val_samples=validation_samples)

Epoch 1/5
1887/1887 [==============================] - 126s - loss: 1.4154 - acc: 0.5670 - val_loss: 0.2495 - val_acc: 0.9108

In [26]:
model.save_weights('vgg_plant.h5')

In [56]:
model.evaluate_generator(validation_generator, validation_samples)

[0.089699285288891165, 0.97228915777551117]

In [18]:
last_conv_index = 0

for i, l in enumerate(model.layers):
    if type(l) is Convolution2D:
        last_conv_index = i

last_conv_index

30

In [21]:
all_conv_layers = model.layers[:last_conv_index+1]
fully_connected_layers = model.layers[last_conv_index+1:]

In [22]:
fully_conv_model = Sequential(all_conv_layers)

In [23]:
train_features = fully_conv_model.predict_generator(train_generator, training_samples)
validation_features = fully_conv_model.predict_generator(validation_generator, validation_samples)

In [25]:
train_labels = np_utils.to_categorical(train_generator.classes)
validation_labels = np_utils.to_categorical(validation_generator.classes)

In [29]:
fully_conv_model.layers[-1].output_shape[1:]

(14, 14, 512)

In [51]:
new_model = Sequential([
    MaxPooling2D((2, 2), strides=(2, 2), input_shape=fully_conv_model.layers[-1].output_shape[1:]),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

In [52]:
new_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [53]:
new_model.fit(train_features, train_labels, batch_size=64, nb_epoch=5, 
              validation_data=(validation_features, validation_labels))

Train on 1887 samples, validate on 830 samples
Epoch 1/5
1887/1887 [==============================] - 5s - loss: 10.1352 - acc: 0.3630 - val_loss: 8.3503 - val_acc: 0.4819

In [54]:
model.optimizer.lr = 1e-4
new_model.fit(train_features, train_labels, batch_size=64, nb_epoch=5, 
              validation_data=(validation_features, validation_labels))

Train on 1887 samples, validate on 830 samples
Epoch 1/5
1887/1887 [==============================] - 5s - loss: 10.1304 - acc: 0.3715 - val_loss: 8.3503 - val_acc: 0.4819

In [55]:
model.optimizer.lr = 0.01
new_model.fit(train_features, train_labels, batch_size=64, nb_epoch=5, 
              validation_data=(validation_features, validation_labels))

Train on 1887 samples, validate on 830 samples
Epoch 1/5
1887/1887 [==============================] - 5s - loss: 10.1304 - acc: 0.3715 - val_loss: 8.3503 - val_acc: 0.4819

KeyboardInterrupt: 

In [50]:
model.optimizer.lr = 1e-5
new_model.fit(train_features, train_labels, batch_size=64, nb_epoch=5, 
              validation_data=(validation_features, validation_labels))

Train on 1887 samples, validate on 830 samples
Epoch 1/5
1887/1887 [==============================] - 1s - loss: 0.6694 - acc: 0.7165 - val_loss: 1.7462 - val_acc: 0.3602

In [ ]:
# model.fit_generator(train_generator, epochs=5, steps_per_epoch=training_samples//32, 
#                     validation_data=validation_generator, validation_steps=validation_samples//32)

Epoch 1/5


In [9]:
model.load_weights('AH10_1.h5')

In [23]:
from PIL import Image

def convert(img):
    img = Image.open(img)
    img = img.resize((224,224), Image.ANTIALIAS)
    img = np.expand_dims(np.array(img), 0)
    return img

In [24]:
def predict_image(img):
    return np.argmax(model.predict(img))

image = convert('41.jpg')
print predict_image(image)

2
